# Notebook for comparing the impact of the CTI correction on the covariances.

Initially written 14 Oct 2022 by Craig Lage.

In [ ]:
import sys, os, glob, time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import astropy.io.fits as pf
from lsst.daf.butler import Butler

In [ ]:
exposure=3021120600576
runA = "u/cslage/bps_13144S"
runB = "u/cslage/sdf/BOT/ptc_13144_11_20221014"

A_Butler = Butler("/repo/main", collections=[runA])
B_Butler = Butler("/repo/main", collections=[runB])
camera = A_Butler.get('camera', instrument='LSSTCam')
nameA = 'FULLCOV-No CTI'
nameB = 'FULLCOV-With CTI'
DETECTOR=11
A_PTC = A_Butler.get('ptc', detector=DETECTOR, exposure=exposure, instrument='LSSTCam')
B_PTC = B_Butler.get('ptc', detector=DETECTOR, exposure=exposure, instrument='LSSTCam')
ptcs = [A_PTC, B_PTC]
names = [nameA, nameB]
colors=['green', 'orange']
markers=['x','+']

In [ ]:
# Next plot the covariance vs flux 
RAFT='R02'
SENSOR='S02'
VENDOR='ITL'
PlotDelta = 3
pdf = PdfPages("/sdf/group/rubin/u/cslage/BOT_LSSTCam/cti/Covariance_vs_Flux_%s_14Oct22.pdf"%(DETECTOR))

for ampObject in camera[0].getAmplifiers():
    amp = ampObject.getName()
    fig = plt.figure(figsize = (16,8))
    plt.suptitle("Covariance vs Flux - Amp %s, %s-%s"%(amp,RAFT,SENSOR), fontsize = 24)
    plt.subplots_adjust(wspace=0.3, hspace=0.6)
    for m, ptc in enumerate(ptcs):
        plotCounter = 0
        means = ptc.finalMeans
        xCorrs = ptc.covariances
        gains = ptc.gain
        gain = gains[amp]
        NumFluxes = int(len(means[amp]))
        serialCov = np.zeros([NumFluxes,PlotDelta])
        for jj in range(PlotDelta-1, -1, -1):
            for ii in range(PlotDelta):
                plotCounter += 1
                plt.subplot(PlotDelta, PlotDelta, plotCounter)
                cov = []
                flux = []

                for n in range(NumFluxes):
                    xcorr = xCorrs[amp][n][ii][jj]
                    mean = means[amp][n]
                    if ii == 0 and jj == 0:
                        xcorr = xcorr - mean / gain
                        cov.append(-xcorr)
                    else:
                        cov.append(xcorr)
                    flux.append(mean)
                cov = np.array(cov)
                flux = np.array(flux)

                plt.scatter(flux, cov, color=colors[m], marker=markers[m], s=50, label=names[m])
                coefs = np.polyfit(flux*flux, cov, 1)
                if jj == 1:
                    serialCov[ampNum, ii] = coefs[0]
                xplot = np.linspace(0,150000, 100)
                yplot = max(0, coefs[0])*xplot*xplot
                plt.plot(xplot,yplot, color = colors[m], lw = 2, ls = '--')
                plt.title("Pixel: (%d, %d)"%(ii, jj), fontsize = 12)
                if jj == 0:
                    plt.xlabel("Central Pixel Charge(e-)", fontsize = 12)
                if ii == 0:
                    plt.ylabel("Correlation", fontsize = 12)
                plt.xlim(0,80000)
                plt.xticks([0,50000],fontsize = 12)

                if ii == 0 and jj == 0:
                    plt.ylim(-1000,10000)
                elif ii == 0 and jj == 1:
                    plt.ylim(-20,1500)
                elif ii == 1 and jj == 0:
                    plt.ylim(-20,750)
                elif ii == 2 and jj == 0:
                    plt.ylim(-20,200)
                elif ii == 1 and jj < 2:
                    plt.ylim(-20,400)
                else:
                    plt.ylim(-20,200)
                plt.legend(loc='upper left')

    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()